In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [3]:
GAP = nn.Sequential(
            nn.GroupNorm(16, 768),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool3d((1,1,1)),
            nn.Conv3d(768, 256, kernel_size=1, stride=1, padding=0)
        )

In [4]:
GAP

Sequential(
  (0): GroupNorm(16, 768, eps=1e-05, affine=True)
  (1): ReLU(inplace=True)
  (2): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (3): Conv3d(768, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

In [7]:
dec4 = torch.randn(1, 768, 3, 3, 3)


In [8]:
dec4.shape

torch.Size([1, 768, 3, 3, 3])

In [9]:
x_feat = GAP(dec4) 

In [10]:
x_feat.shape

torch.Size([1, 256, 1, 1, 1])

In [11]:
out = torch.randn(1, 48, 96, 96, 96)

In [30]:
GAP2 = nn.Sequential(
    nn.GroupNorm(16, 48),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool3d((1, 1, 1)),
    nn.Conv3d(48, 68, kernel_size=1, stride=1, padding=0),
    nn.Conv3d(68, 112, kernel_size=1, stride=1, padding=0),
    nn.Conv3d(112, 256, kernel_size=3, stride=1, padding=1), 
    nn.ReLU(inplace=True),
    nn.Conv3d(256, 256, kernel_size=3, stride=1, padding=1),  
    nn.ReLU(inplace=True)
)

In [31]:
x_feat_dec = GAP2(out)

In [32]:
x_feat_dec.shape

torch.Size([1, 256, 1, 1, 1])

In [21]:
GAP3 = nn.Sequential(
            nn.Conv3d(256, 1, kernel_size=1, stride=1, padding=0),
            nn.Conv3d(256, 1, kernel_size=1, stride=1, padding=0),
            nn.Conv3d(256, 1, kernel_size=1, stride=1, padding=0)
        )

In [22]:
x = (x_feat + x_feat_dec)

In [24]:
result_matrix = torch.mean(torch.stack([x_feat, x_feat_dec], dim=0), dim=0)

In [25]:
result_matrix.shape

torch.Size([1, 256, 1, 1, 1])

In [27]:
import numpy as np

# Ma trận đầu vào
input_matrix = np.array([[-0.3666, -0.1789, -0.3015],
                         [-1.1863,  0.8609, -0.9325],
                         [ 0.6061, -0.2307,  1.0710]])

# Hàm sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Tính toán sigmoid cho ma trận đầu vào
output_matrix = sigmoid(input_matrix)

print(output_matrix)


[[0.40936284 0.45539391 0.42519084]
 [0.23392133 0.70284866 0.28241779]
 [0.64705065 0.44257945 0.74478704]]


In [1]:
import torch
import torch.nn as nn

In [10]:
# Định nghĩa mạng tích chập để chuyển đổi kích thước tensor
class Conv3DNet(nn.Module):
    def __init__(self):
        super(Conv3DNet, self).__init__()
        self.conv1 = nn.Conv3d(768, 512, kernel_size=3, padding=1, stride=1)
        self.conv2 = nn.Conv3d(512, 256, kernel_size=3, padding=1, stride=2)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.activation(self.conv2(x))
        return x

In [9]:
conv3d_net = Conv3DNet()

# Tạo một tensor đầu vào có kích thước (1, 768, 3, 3, 3)
input_tensor = torch.randn(1, 768, 3, 3, 3)

# Sử dụng mạng tích chập để chuyển đổi kích thước tensor
output_tensor = conv3d_net(input_tensor)

print(output_tensor.shape)  # In ra kích

torch.Size([1, 512, 3, 3, 3])
